In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
demo = gpd.read_file("2010demo.json")

In [7]:
precinct = gpd.read_file("all_states_precincts_neighbor.json")

In [8]:
demoCopy = demo.copy()
precinctCopy = precinct.copy()

In [9]:
demo.head(1)

,GEOID10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,DP0010001,DP0010002,DP0010003,DP0010004,...,DP0210001,DP0210002,DP0210003,DP0220001,DP0220002,DP0230001,DP0230002,Shape_Leng,Shape_Area,geometry
0,01005950300,Census Tract 9503,352811329,499970,+31.7908933,-085.5670514,1638,98,87,88,...,673,544,129,1283,355,2.36,2.75,1.164633,0.033648,"POLYGON ((-85.43637 31.78898, -85.43676 31.788..."


In [10]:
demo = demoCopy.copy()
demoColumns = list(demo.columns)

In [ ]:
keep = ["DP0080001","DP0080002","DP0080003"
        ,"DP0080004","DP0080005","DP0080006","DP0080007","DP0080008","DP0080009","DP0080010"
        ,"DP0080011","DP0080012","DP0080013","DP0080014","DP0080015","DP0080016","DP0080017"
        ,"DP0080018","DP0080019","DP0080020","DP0080021","DP0080022","DP0080023","DP0080024", "DP0110002"]

rename = ["Total:","Population of one race:"
          ,"White","Black or African American","American Indian and Alaska Native"
          ,"Asian:","Asian Indian","Chinese","Filipino","Japanese","Korean","Vietnamese"
          ,"Other Asian","Native Hawaiian and Other Pacific Islander:","Native Hawaiian"
          ,"Guamanian or Chamorro","Samoan","Other Pacific Islander","Some Other Race"
          ,"Population of Two or More Races","White; American Indian and Alaska Native"
          ,"White; Asian","White; Black or African American","White; Some Other Race", "Hispanic or Latino (of any race):"]

In [51]:
mapper = {}
for i in range(len(keep)):
    mapper[keep[i]] = rename[i]
    
new_keep = ["Total:", 'White', 'Black or African American',
            'American Indian and Alaska Native','Asian:']
m = mapper.copy()
for key, value in m.items():
    if value not in new_keep:
        del mapper[key]
mapper

{'DP0080001': 'Total:',
 'DP0080003': 'White',
 'DP0080004': 'Black or African American',
 'DP0080005': 'American Indian and Alaska Native',
 'DP0080006': 'Asian:'}

In [52]:
mapper["GEOID10"] = "GEOID10"
mapper["geometry"] = "geometry"
for key in mapper.keys():
    demoColumns.remove(key)

In [53]:
demo.drop(demoColumns, axis=1, inplace=True)

In [54]:
demo.rename(mapper, axis=1, inplace=True)

In [57]:
demo["Other"] = 0
for index, row in demo.iterrows():
    demo.at[index, "Other"] = row["Total:"]-row["White"]-row["Black or African American"]-row["American Indian and Alaska Native"] - row["Asian:"]

In [58]:
demoDroppedCopy = demo.copy()
demo.head(1)

,GEOID10,Total:,White,Black or African American,American Indian and Alaska Native,Asian:,geometry,Other
0,01005950300,1638,718,864,4,0,"POLYGON ((-85.43637 31.78898, -85.43676 31.788...",52


# start deleting rows that not in state we want

In [61]:
RI = precinct[precinct["State"] == "ri"]
VA = precinct[precinct["State"] == "va"]
TX = precinct[precinct["State"] == "tx"]
RIcopy = RI.copy()
VAcopy = VA.copy()
TXcopy = TX.copy()

In [65]:
census2state = {}
states = {"ri":RI, "va":VA, "tx":TX}

In [81]:
import time

for stateID, stateDF in states.items():
    t1 = time.time()
    demo = demoDroppedCopy.copy()
    print("Start Mapping:")
    counter = 0
    demo["belongsTo"] = False
    for index, row in demo.iterrows():
        if len(stateDF[stateDF["geometry"].intersects(row["geometry"])]) != 0:
            demo.at[index, "belongsTo"] = True
            print("@",end="")
        else:
            if counter%1000 == 0:
                print("#",end="")
        counter+=1
    t2 = time.time()
    print("Time:", (t2-t1)/60, "Minutes")
    demoStateDF = demo[demo["belongsTo"] == True]
    census2state[stateID] = demoStateDF

Start Mapping:
##############@@@@@@@##################@@@@@@@@@@@@@@@#@@@@##########################@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@################Time: 4.642427198092142 Minutes
Start Mapping:
##############@@#@@@@@@#############@@@@@@@@@#@@@@@@@#@#@@@@@@@@@@@@@@@@@@@@@@##################@@@@@@#@@@@@@@@@@@@@@@@@@@@@@@@@#@@@@@@@@@@@##########@@@@@@@@@@@@@@@#@@######@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [95]:
#for key, df in census2state.items():
#    df.to_file(key.upper() + "_demo2010.json", driver="GeoJSON")

In [96]:
merged = pd.concat(list(census2state.values()))

In [97]:
merged.head(1)

,GEOID10,Total:,White,Black or African American,American Indian and Alaska Native,Asian:,geometry,Other,belongsTo
13108,09011707100,5297,4951,57,61,75,"POLYGON ((-71.94327 41.43365, -71.94319 41.433...",153,True


In [98]:
#merged.to_file("merged_unclean_demo.json", driver="GeoJSON")